# Getting the relation between baroclinic streamfunction and cross-stream distance from the GLORYS12V1 data

In [1]:
import pickle
import numpy as np
import xarray as xr
from scipy.io import loadmat
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import interp1d

import gsw
import csaps

import warnings

## Sample points along hydrographic section transect

In [2]:
# read in hydrographic section data
data = loadmat('data/SR1b_section/sr1b_all.mat')

icyc = np.where(data['buf_year'] != 1999)[1] # exclude 1999 # TODO: why?

buf_lat = data['buf_lat'][0,icyc]
buf_lon = data['buf_lon'][0,icyc]
buf_sal = data['buf_sal'][:,icyc]
buf_temp = data['buf_temp'][:,icyc]
buf_press = data['buf_press'].reshape(3000)
buf_year = data['buf_year'][:,icyc].reshape(316)

In [3]:
# sample points along section
start_lon_vals = []
end_lon_vals = []
start_lat_vals = []
end_lat_vals = []
for yr in np.unique(buf_year):
  idx_yr = np.where(buf_year == yr)[0]
  start_lon_vals.append(np.min(buf_lon[idx_yr]))
  end_lon_vals.append(np.max(buf_lon[idx_yr]))
  start_lat_vals.append(np.max(buf_lat[idx_yr]))
  end_lat_vals.append(np.min(buf_lat[idx_yr]))
start_lon = np.mean(np.array(start_lon_vals))
end_lon = np.mean(np.array(end_lon_vals))
start_lat = np.mean(np.array(start_lat_vals))
end_lat = np.mean(np.array(end_lat_vals))

# Extract unique points along the section
nr_points = 30
lon_along_section = np.linspace(start_lon, end_lon, nr_points)
lat_along_section = np.linspace(start_lat, end_lat, nr_points)

# calculate distance from southern end of section
dist_along_section = np.zeros(len(lat_along_section))
for i in range(len(dist_along_section)):
  dist_along_section[i] = gsw.distance(np.mean(lon_along_section)*np.ones(2), np.array([min(lat_along_section), lat_along_section[i]]))[0]

## Process the CMEMS data

In [4]:
# Read in data
cmems = xr.open_dataset('data/CMEMS_reanalysis/cmems_mod_glo_phy_my_0.083deg_P1M-m_2010-2012.nc') # downloaded on 07-07-2025
cmems_lat = cmems['latitude'].values
cmems_lon = cmems['longitude'].values
cmems_depth = cmems['depth'].values
cmems_time = cmems['time'].values
idx = np.arange(len(cmems_time))

cmems_so = xr.DataArray(cmems['so'].values, dims=['time', 'depth', 'latitude', 'longitude'],
                        coords={'time': cmems_time,
                                'longitude': cmems_lon,
                                'latitude': cmems_lat,
                                'depth': cmems_depth})

cmems_thetao = xr.DataArray(cmems['thetao'].values, dims=['time', 'depth', 'latitude', 'longitude'],
                            coords={'time': cmems_time,
                                    'longitude': cmems_lon,
                                    'latitude': cmems_lat,
                                    'depth': cmems_depth})

In [5]:
# transform depth to pressure
cmems_p = np.zeros((len(cmems_depth), len(cmems_lat)))
for i in range(len(cmems_lat)):
        cmems_p[:,i] = gsw.p_from_z(-cmems_depth, cmems_lat[i])

# interpolate p to section points
p_section_cmems = np.zeros((len(cmems_depth), len(lat_along_section)))
for i in range(len(cmems_depth)):
    p_section_cmems[i,:] = np.interp(lat_along_section, cmems_lat, cmems_p[i,:])

In [6]:
# calculate absolute salinity and conservative temperature
# cmems_SA = np.zeros(np.shape(cmems_so))
# for t in range(len(cmems_time)):
#     for d in range(len(cmems_depth)):
#         for i in range(len(cmems_lat)):
#             for j in range(len(cmems_lon)):
#                 cmems_SA[t,d,i,j] = gsw.SA_from_SP(cmems_so[t,d,i,j], cmems_p[d,i], cmems_lon[j], cmems_lat[i])
# np.save('data/cmems_SA_1993-2005.npy', cmems_SA)  # save pre-calculated SA
#cmems_SA = np.load('data/CMEMS_reanalysis/cmems_SA_2010-2012.npy') # load pre-calculated SA
cmems_SA = xr.DataArray(np.load('data/CMEMS_reanalysis/cmems_SA_2010-2012.npy'), 
                        dims=['time', 'depth', 'latitude', 'longitude'],
                        coords={'time': cmems_time,
                                'longitude': cmems_lon,
                                'latitude': cmems_lat,
                                'depth': cmems_depth})
cmems_CT = gsw.CT_from_pt(cmems_SA, cmems_thetao)

In [7]:
# interpolate SA and CT to section points
SA_section_cmems = np.zeros((len(idx), len(cmems_depth), len(lat_along_section)))
CT_section_cmems = np.zeros((len(idx), len(cmems_depth), len(lat_along_section)))
points = np.array([(lat, lon) for lat,lon in zip(lat_along_section,lon_along_section)])
for i in range(len(idx)):
    for j in range(len(cmems_depth)):
        f_SA_cmems = RegularGridInterpolator((cmems_lat, cmems_lon), cmems_SA.values[i,j,:,:])
        f_CT_cmems = RegularGridInterpolator((cmems_lat, cmems_lon), cmems_CT.values[i,j,:,:])
        SA_section_cmems[i,j,:] = f_SA_cmems(points)
        CT_section_cmems[i,j,:] = f_CT_cmems(points)

# interpolate in p (to high resolution)
SA_section_interp_cmems = np.zeros((len(idx), len(buf_press), len(lat_along_section)))
CT_section_interp_cmems = np.zeros((len(idx), len(buf_press), len(lat_along_section)))
for i in range(len(idx)):
    for j in range(len(lat_along_section)):
        f_SA_cmems = interp1d(p_section_cmems[:,j], SA_section_cmems[i,:,j], bounds_error=False, fill_value=np.nan)
        f_CT_cmems = interp1d(p_section_cmems[:,j], CT_section_cmems[i,:,j], bounds_error=False, fill_value=np.nan)
        SA_section_interp_cmems[i,:,j] = f_SA_cmems(buf_press)
        CT_section_interp_cmems[i,:,j] = f_CT_cmems(buf_press)

In [29]:
# compute baroclinic streamfunction
gpan_cmems = np.zeros((len(idx), len(buf_press), len(lat_along_section)))
for i in range(len(idx)):
    gpan_cmems[i,:,:] = -gsw.geo_strf_dyn_height(SA_section_interp_cmems[i,:,:], CT_section_interp_cmems[i,:,:], buf_press, 1500)

cmems_phi1500 = abs(gpan_cmems[:,249,:])

# make spline fit through relationship between phi1500 and distance
dist_repeat = np.tile(dist_along_section, len(cmems_time))
cmems_phi1500_flatten = cmems_phi1500.flatten()
pp_dist_cmems = csaps.csaps(cmems_phi1500_flatten[np.argsort(cmems_phi1500_flatten)[:-np.count_nonzero(np.isnan(cmems_phi1500_flatten))]],
                      dist_repeat[np.argsort(cmems_phi1500_flatten)[:-np.count_nonzero(np.isnan(cmems_phi1500_flatten))]],
                      smooth = 0.85)

# save the spline fit
with open('data/CMEMS_reanalysis/pp_dist_cmems.pkl', 'wb') as f:
    pickle.dump(pp_dist_cmems, f)

/nethome/5664187/miniconda3/envs/carlo/lib/python3.13/site-packages/csaps/_sspumv.py:301: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  u = la.spsolve(a, b)
